In [1]:
import ee
import geemap
import os
import numpy as np
import geopandas as gpd

In [2]:
os.getcwd()

'G:\\OneDrive\\casa0010dissertation\\00_06 jaipur code\\data_process'

In [3]:
import os
os.chdir("G:/OneDrive/casa0010dissertation/00_06 jaipur code")


In [4]:
ee.Authenticate()

True

In [5]:
ee.Initialize()

In [25]:
Map = geemap.Map()
Map.set_center(75.7873, 26.9124, 10)
Map

Map(center=[26.9124, 75.7873], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

In [8]:
# Define AOI
# Load the ward dataset

url = "data/raw/JMC_all_v2.geojson"
wards = gpd.read_file(url)
# Check the current CRS
print("Original CRS:", wards.crs)
print("This is UTM Zone 43N for India - perfect for Jaipur!")

Original CRS: EPSG:32643
This is UTM Zone 43N for India - perfect for Jaipur!


In [9]:
# Convert to Earth Engine FeatureCollection
# geemap.geopandas_to_ee() will automatically handle the CRS conversion to WGS84
ee_wards = geemap.geopandas_to_ee(wards, geodesic=False)

In [10]:
# Add filtered wards to map
Map.addLayer(
ee_wards,
{ 'color': "1e90ff", 'width': 2, 'fillColor': "00000000" },
"Jaipur Wards"
)

In [11]:
# Get the geometry for AOI (union of all wards)
aoi = ee_wards.geometry()
print("AOI defined from Jaipur wards GeoJSON")

AOI defined from Jaipur wards GeoJSON


In [12]:
# Load GHSL Built-up Characteristics dataset
# GHSL data is from 2018 (latest available)
ghsl = ee.Image("JRC/GHSL/P2023A/GHS_BUILT_C/2018").select('built_characteristics').clip(aoi)

print("GHSL dataset loaded (2018 data)")

GHSL dataset loaded (2018 data)


In [13]:
# Load VIIRS Nighttime Lights dataset
viirs_collection = ee.ImageCollection('NOAA/VIIRS/DNB/ANNUAL_V22') \
                     .filter(ee.Filter.date('2023-01-01', '2024-01-01')) \
                     .filterBounds(aoi)

viirs = viirs_collection.select('average').first().clip(aoi)

print("VIIRS dataset loaded (2023 annual data)")

VIIRS dataset loaded (2023 annual data)


In [14]:
# Create residential mask (classes 11-15 are residential buildings)
residential_mask = ghsl.gte(11).And(ghsl.lte(15))
print("Residential mask created (GHSL classes 11-15)")

Residential mask created (GHSL classes 11-15)


In [15]:
# Map GHSL classes to building heights
# Class 11: ≤3m -> 1.5m average
# Class 12: 3-6m -> 4.5m average  
# Class 13: 6-15m -> 10.5m average
# Class 14: 15-30m -> 22.5m average
# Class 15: >30m -> 35m average
height_map = ee.Image(0) \
    .where(ghsl.eq(11), 1.5) \
    .where(ghsl.eq(12), 4.5) \
    .where(ghsl.eq(13), 10.5) \
    .where(ghsl.eq(14), 22.5) \
    .where(ghsl.eq(15), 35)
print("Height mapping completed")

Height mapping completed


In [16]:
# Calculate residential volume (height * pixel area)
pixel_area = 100  # 10m x 10m pixels = 100 m²
residential_volume = height_map.multiply(pixel_area).updateMask(residential_mask)
print("Residential volume calculated (m³)")

Residential volume calculated (m³)


In [17]:
# Define illumination threshold
# Simple binary classification: any light detected (>0) vs no light (=0)
illumination_threshold = 0
print("Using simple >0 threshold for illumination detection")

Using simple >0 threshold for illumination detection


In [18]:
# Create illumination mask
# Earth Engine will automatically handle projection when combining datasets
illuminated_mask = viirs.gt(illumination_threshold)
print("Illumination mask created")

Illumination mask created


In [19]:
# Create Layer 1 - Illuminated residential volume
illuminated_residential_volume = residential_volume.updateMask(
    residential_mask.And(illuminated_mask)
).rename('illuminated_residential_volume')

print("Layer 1 created: Illuminated residential volume")

# Create Layer 2 - Unilluminated residential volume
unilluminated_residential_volume = residential_volume.updateMask(
    residential_mask.And(illuminated_mask.Not())
).rename('unilluminated_residential_volume')

print("Layer 2 created: Unilluminated residential volume")

Layer 1 created: Illuminated residential volume
Layer 2 created: Unilluminated residential volume


In [20]:
# Create Layer 3 - Nighttime light intensity of residential areas
residential_light_intensity = viirs.updateMask(
    residential_mask
).rename('residential_light_intensity')

print("Layer 3 created: Residential light intensity")

Layer 3 created: Residential light intensity


In [21]:
# Define visualization parameters
ghsl_vis = {
    'min': 1, 'max': 25,
    'palette': [
        '#718c6c',  # 1: open spaces, low vegetation
        '#8ad86b',  # 2: open spaces, medium vegetation  
        '#c1ffa1',  # 3: open spaces, high vegetation
        '#01b7ff',  # 4: open spaces, water
        '#ffd501',  # 5: open spaces, road
        '#d28200',  # 11: residential ≤3m
        '#fe5900',  # 12: residential 3-6m
        '#ff0101',  # 13: residential 6-15m  
        '#ce001b',  # 14: residential 15-30m
        '#7a000a',  # 15: residential >30m
        '#ff9ff4',  # 21: non-residential ≤3m
        '#ff67e4',  # 22: non-residential 3-6m
        '#f701ff',  # 23: non-residential 6-15m
        '#a601ff',  # 24: non-residential 15-30m  
        '#6e00fe'   # 25: non-residential >30m
    ]
}

# Volume visualization - Blue gradient (light to dark)
volume_vis = {
    'min': 150, 'max': 3500,  # Will be updated after checking data range
    'palette': [
        '#f7fbff',  # Very light blue
        '#deebf7',  # Light blue
        '#c6dbef',  # 
        '#9ecae1',  # 
        '#6baed6',  # Medium blue
        '#4292c6',  # 
        '#2171b5',  # 
        '#08519c',  # 
        '#08306b'   # Dark blue
    ]
}

# Light intensity visualization - Red gradient (light to dark)
light_vis = {
    'min': 2, 'max': 79,  # Will be updated after checking data range
    'palette': [
        '#fff5f0',  # Very light red
        '#fee0d2',  # Light red
        '#fcbba1',  # 
        '#fc9272',  # 
        '#fb6a4a',  # Medium red
        '#ef3b2c',  # 
        '#cb181d',  # 
        '#a50f15',  # 
        '#67000d'   # Dark red
    ]
}

print("Visualization parameters defined with monochromatic gradients")


Visualization parameters defined with monochromatic gradients


In [27]:
Map.addLayer(ghsl, ghsl_vis, 'GHSL Built Characteristics')
Map.addLayer(viirs, light_vis, 'VIIRS Nighttime Lights')
Map.addLayer(illuminated_residential_volume, volume_vis, 'Illuminated Residential Volume')
Map.addLayer(unilluminated_residential_volume, volume_vis, 'Unilluminated Residential Volume') 
Map.addLayer(residential_light_intensity, light_vis, 'Residential Light Intensity')

print("Layers added to map")

# Step 17: Add legend
legend_dict = {
    'Open spaces': '#718c6c',
    'Residential ≤3m': '#d28200',
    'Residential 3-6m': '#fe5900', 
    'Residential 6-15m': '#ff0101',
    'Residential 15-30m': '#ce001b',
    'Residential >30m': '#7a000a',
    'Non-residential': '#ff9ff4'
}

Map.add_legend(title="GHSL Building Classes", legend_dict=legend_dict)
print("Legend added")

# Step 18: Display the map
Map

Layers added to map
Legend added


Map(bottom=55754.0, center=[26.672039413635783, 75.90362438569473], controls=(WidgetControl(options=['position…

In [26]:
# Step 22: Check data ranges for all 5 layers and update visualization parameters
print("\n=== CHECKING DATA RANGES FOR ALL 5 LAYERS ===")

# 1. GHSL Built Characteristics
print("1. GHSL Built Characteristics:")
ghsl_range = ghsl.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e8
)
print("   Range:", ghsl_range.getInfo())

# 2. VIIRS Nighttime Lights
print("2. VIIRS Nighttime Lights:")
viirs_range = viirs.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=500,
    maxPixels=1e8
)
print("   Range:", viirs_range.getInfo())

# 3. Illuminated Residential Volume
print("3. Illuminated Residential Volume:")
illum_volume_range = illuminated_residential_volume.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e8
)
print("   Range:", illum_volume_range.getInfo())

# 4. Unilluminated Residential Volume
print("4. Unilluminated Residential Volume:")
unillum_volume_range = unilluminated_residential_volume.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e8
)
print("   Range:", unillum_volume_range.getInfo())

# 5. Residential Light Intensity
print("5. Residential Light Intensity:")
res_light_range = residential_light_intensity.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e8
)
print("   Range:", res_light_range.getInfo())

print("\n=== USE THESE RANGES TO UPDATE VISUALIZATION PARAMETERS ===")
print("Copy the max values above and update the 'max' values in volume_vis and light_vis")


=== CHECKING DATA RANGES FOR ALL 5 LAYERS ===
1. GHSL Built Characteristics:
   Range: {'built_characteristics_max': 24, 'built_characteristics_min': 0}
2. VIIRS Nighttime Lights:
   Range: {'average_max': 78.00666046142578, 'average_min': 1.9543238878250122}
3. Illuminated Residential Volume:
   Range: {'illuminated_residential_volume_max': 3500, 'illuminated_residential_volume_min': 150}
4. Unilluminated Residential Volume:
   Range: {'unilluminated_residential_volume_max': None, 'unilluminated_residential_volume_min': None}
5. Residential Light Intensity:
   Range: {'residential_light_intensity_max': 78.00666046142578, 'residential_light_intensity_min': 2.2137434482574463}

=== USE THESE RANGES TO UPDATE VISUALIZATION PARAMETERS ===
Copy the max values above and update the 'max' values in volume_vis and light_vis


In [24]:
# # Create output directory
# import os
# out_dir = 'data/cleaned'
# os.makedirs(out_dir, exist_ok=True)

# print("Downloading layers to data/cleaned...")

# # Download the 3 smaller files directly
# print("Downloading smaller files directly...")

# # 1. Download GHSL Built Characteristics (10m) - Direct download
# geemap.ee_export_image(
#     ghsl,
#     filename=f'{out_dir}/GHSL_Built_Characteristics.tif',
#     scale=10,
#     crs='EPSG:32643',
#     region=aoi
# )
# print("✓ GHSL Built Characteristics downloaded (10m)")

# # 2. Download VIIRS Nighttime Lights (464m) - Direct download
# geemap.ee_export_image(
#     viirs,
#     filename=f'{out_dir}/VIIRS_Nighttime_Lights.tif',
#     scale=464,
#     crs='EPSG:32643',
#     region=aoi
# )
# print("✓ VIIRS Nighttime Lights downloaded (464m)")

# # 3. Download Residential Light Intensity (464m) - Direct download
# geemap.ee_export_image(
#     residential_light_intensity,
#     filename=f'{out_dir}/Residential_Light_Intensity.tif',
#     scale=464,
#     crs='EPSG:32643',
#     region=aoi
# )
# print("✓ Residential Light Intensity downloaded (464m)")

# # 4. Export Illuminated Residential Volume to Google Drive (too large for direct download)
# print("Exporting Illuminated Residential Volume to Google Drive (too large for direct download)...")

# task_volume = ee.batch.Export.image.toDrive(
#     image=illuminated_residential_volume,
#     description='Illuminated_Residential_Volume',
#     scale=10,
#     region=aoi,
#     crs='EPSG:32643',
#     maxPixels=1e13,
#     folder='Jaipur'
# )

# task_volume.start()
# print("✓ Illuminated Residential Volume export task started")
# print("Check Google Drive folder: GHSL_VIIRS_Jaipur")
# print("Download manually to data/cleaned/ when export completes")

# print(f"Direct downloads completed in: {out_dir}/")
# print("Files ready:")
# print("- GHSL_Built_Characteristics.tif")
# print("- VIIRS_Nighttime_Lights.tif") 
# print("- Residential_Light_Intensity.tif")
# print("Pending: Illuminated_Residential_Volume.tif (via Google Drive)")